In [69]:
import pymorphy2 as pm2 
pmm = pm2.MorphAnalyzer() 
from pymystem3 import Mystem
m = Mystem()
import re
import os

In [70]:
def get_all_verbs(text):
    verbs = {}
    n_text = re.sub(r'[^\w\s]','', text) 
    n_text = re.sub(r'\d', '', n_text) 
    n_text = re.sub(r'[A-Za-z]', '', n_text)
    print('start analysing')
    ana = m.analyze(n_text)
    print('analyse is ready')
    for word in ana:
        if 'analysis' in word:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            if pos == 'V':
                verb = word['analysis'][0]['lex']
                print(verb)
                if verb in verbs:
                    verbs[verb] += 1
                else:
                    verbs[verb] = 1
    return verbs

In [68]:
with open('test.txt', 'r', encoding='utf-8') as t:
    text = t.read()
verbs = get_all_verbs(text)

Геометрия  неотъемлемая составляющая общей культуры а геометрические методы служат одним из инструме
start analysing


KeyboardInterrupt: 

In [4]:
verbs

{'базироваться': 1,
 'быть': 2,
 'включать': 1,
 'владеть': 2,
 'вносить': 1,
 'возникать': 1,
 'вселять': 1,
 'выполнять': 1,
 'выражаться': 1,
 'вычислять': 1,
 'говорить': 1,
 'давать': 1,
 'доходить': 1,
 'жить': 1,
 'измерять': 4,
 'изображать': 1,
 'изучаться': 1,
 'иметь': 3,
 'исчерпывать': 1,
 'лежать': 2,
 'мочь': 1,
 'накоплять': 2,
 'находиться': 1,
 'начинать': 1,
 'носить': 1,
 'обладать': 1,
 'обусловливать': 1,
 'означать': 1,
 'оказываться': 1,
 'определяться': 1,
 'основываться': 1,
 'осуществлять': 1,
 'относиться': 1,
 'подтвердить': 1,
 'подтверждать': 1,
 'позволять': 2,
 'поражать': 1,
 'представлять': 1,
 'приводить': 1,
 'проводить': 1,
 'продвигать': 1,
 'происходить': 1,
 'располагать': 1,
 'решать': 2,
 'свидетельствовать': 1,
 'связывать': 1,
 'служить': 5,
 'совершать': 1,
 'создавать': 1,
 'составлять': 1,
 'состоять': 2,
 'сохраняться': 1,
 'способствовать': 3,
 'становиться': 3,
 'считаться': 1,
 'сыграть': 1,
 'требовать': 1,
 'убеждать': 1,
 'характер

In [71]:
def get_tfs(verbs):
    tfs = []
    v_total = sum(verbs.values())
    for key, value in verbs.items():
        tf = value/v_total
        tfs.append((key, tf))
    return tfs

In [32]:
print(get_tfs(verbs))

[('составлять', 0.011904761904761904), ('служить', 0.05952380952380952), ('способствовать', 0.03571428571428571), ('вселять', 0.011904761904761904), ('считаться', 0.011904761904761904), ('являться', 0.05952380952380952), ('связывать', 0.011904761904761904), ('подтверждать', 0.011904761904761904), ('характеризовать', 0.011904761904761904), ('владеть', 0.023809523809523808), ('продвигать', 0.011904761904761904), ('оказываться', 0.011904761904761904), ('обладать', 0.011904761904761904), ('иметь', 0.03571428571428571), ('определяться', 0.011904761904761904), ('убеждать', 0.011904761904761904), ('относиться', 0.011904761904761904), ('создавать', 0.011904761904761904), ('происходить', 0.011904761904761904), ('измерять', 0.047619047619047616), ('означать', 0.011904761904761904), ('быть', 0.023809523809523808), ('обусловливать', 0.011904761904761904), ('решать', 0.023809523809523808), ('доходить', 0.011904761904761904), ('говорить', 0.011904761904761904), ('находиться', 0.011904761904761904), 

In [72]:
def get_lemmas(article):
    article = re.sub(r'[^\w\s]','', article) 
    article = re.sub(r'\d', '', article) 
    article = re.sub(r'[A-Za-z]', '', article)
    lem_article = [pmm.normal_forms(x)[0] for x in article.split()]
    return lem_article

In [73]:
def get_idfs(verbs, folder):
    f_total = len([name for name in os.listdir(folder) if os.path.isfile(os.path.join(folder, name))]) + 1
    v_presence = {}
    for verb in verbs:
        v_presence[verb] = 1
    idfs = []
    for root, dirs, files in os.walk(folder):
        for name in files:
            if name != 'all geometry.txt': 
                with open(os.path.join(folder, name), 'r', encoding='utf-8') as a:
                    article = a.read()
                lemmas = get_lemmas(article)
                for verb in verbs:
                    if verb in lemmas:
                        v_presence[verb] += 1
    for verb in v_presence:
        idf = f_total/v_presence[verb]
        idfs.append((verb, idf))
    return idfs

In [31]:
print(get_idfs(verbs, 'test collection'))

[('составлять', 4.0), ('служить', 4.0), ('способствовать', 2.0), ('вселять', 4.0), ('считаться', 4.0), ('являться', 2.0), ('связывать', 4.0), ('подтверждать', 4.0), ('характеризовать', 4.0), ('владеть', 4.0), ('продвигать', 4.0), ('оказываться', 4.0), ('обладать', 4.0), ('иметь', 2.0), ('определяться', 4.0), ('убеждать', 4.0), ('относиться', 4.0), ('создавать', 4.0), ('происходить', 4.0), ('измерять', 4.0), ('означать', 4.0), ('быть', 0.8), ('обусловливать', 4.0), ('решать', 2.0), ('доходить', 4.0), ('говорить', 2.0), ('находиться', 2.0), ('требовать', 4.0), ('позволять', 4.0), ('выполнять', 4.0), ('свидетельствовать', 4.0), ('представлять', 4.0), ('вычислять', 4.0), ('возникать', 4.0), ('сохраняться', 4.0), ('поражать', 4.0), ('приводить', 4.0), ('накоплять', 4.0), ('становиться', 2.0), ('вносить', 4.0), ('носить', 4.0), ('подтвердить', 4.0), ('базироваться', 4.0), ('выражаться', 4.0), ('сыграть', 4.0), ('жить', 1.3333333333333333), ('состоять', 4.0), ('совершать', 4.0), ('основыватьс

In [ ]:
def get_tf_idf(verbs):
    #with open('all geometry.txt', 'r', encoding='utf-8') as t:
    #    text = t.read()
    #verbs = get_all_verbs(text)
    tfs = get_tfs(verbs)
    idfs = get_idfs(verbs, 'test collection')
    tf_idfs = []
    for t, i in zip(tfs, idfs):
        tf_idf = t[1] * i[1]
        tf_idfs.append((t[0], tf_idf))
    return tf_idfs

In [ ]:
with open("verbs.json", "w", encoding = 'utf-8') as v:
    json.dump(verbs, v)

In [ ]:
with open("verbs.json", "r", encoding = 'utf-8') as v:
    verbs = json.load(v)

In [ ]:
tf_idfs = get_tf_idf(verbs)
print(tf_idfs)

 Геометрия  неотъемлемая составляющая общей культуры а геометрические методы служат одним из инструм
start analysing


In [ ]:
with open("tf-idf.json", "w", encoding = 'utf-8') as t:
    json.dump(verbs, t)

In [ ]:
with open("tf-idf.json", "r", encoding = 'utf-8') as t:
    verbs = json.load(t)

In [ ]:
with open("verbs.csv", "a", encoding = 'utf-8') as v:
    v.write('verbs;tf-idf')
    for i in tf_idfs:
        v.write(i[0] + ';' + i[1])